In [1]:
%matplotlib inline

In [2]:
import torch
import os
import numpy as np
import pandas as pd
import finetune
import importlib
from tqdm import tqdm
import matplotlib.pyplot as plt

KeyboardInterrupt: 

In [ ]:
# detect_convergence method not finished yet...
def detect_convergence(values):
    steps_to_avg = 3

    for idx, val in enumerate(values):
        # print(val)
        if len(values) - idx <= steps_to_avg:
            print("Finished")
            break

        starting_pos = idx

        avg_last_three_to_six = np.mean(values[starting_pos+steps_to_avg: starting_pos+(steps_to_avg*2)])
        avg_last_three = np.mean(values[starting_pos + steps_to_avg:])
        increase = (avg_last_three - avg_last_three_to_six)/avg_last_three_to_six
        print(f'starting pos: {starting_pos}|increase: {increase}')

In [ ]:
path_to_data = os.path.join('..', '..', 'data')
path_to_models = os.path.join(path_to_data, 'finetuned_models')

datasets = ['FGVC-Aircraft', 'StanfordCars']
model_names = ['resnet', 'densenet', 'vgg']

In [ ]:
# Create inside data directory, a directory for the results export
# The results are split into directories based on their dataset
path_to_results = os.path.join(path_to_data, 'finetuned_model_results')
finetune.safe_mkdir(path_to_results)

for dataset in datasets:
    finetune.safe_mkdir(os.path.join(path_to_results, dataset))

In [ ]:
importlib.reload(finetune)

In [ ]:
importlib.reload(finetune)

# Importing all Resnet models in StanfordCars
models = {}

for dataset in datasets:
    path_to_model_dir = os.path.join(path_to_models, dataset, 'resnet')
    for file_name in os.listdir(path_to_model_dir):
        # print(f'{dataset}: {file} | {path_to_model_dir}')
        name = file_name.split('.')[0]
        print(name)
        full_path = os.path.join(path_to_model_dir, file_name)
        model_saved_state = finetune.load_checkpoint(full_path)
        models[name] = model_saved_state

In [ ]:
for file_name, model in tqdm(models.items()):
    train_hist, train_loss, val_hist = finetune.get_information_from_checkpoint(model)
    dataset = file_name.split('_')[1]
    save_path = os.path.join(path_to_results, dataset)
    content = ''.join([str(value)+'\n' for value in val_hist])
    finetune.write_to_file(save_path+f'/{file_name}.txt', content)

In [ ]:
# For each dataset
# For each architecture get the best model
results = {}

# Read all data from results directory
for dataset in datasets:
    path_to_results_dataset = os.path.join(path_to_results, dataset)
    files = os.listdir(path_to_results_dataset)
    for file_name in files[::-1]:
        if file_name == 'resnet_FGVC-Aircraft_E50_occ0.txt':
            print(file_name)
            name = file_name.split('.')[0]
            p = os.path.join(path_to_results, dataset, file_name)
            content =  finetune.read_file(p)
            values = [float(val) for val in content.split('\n') if val != '']
            results[name] = values
            plt.plot(values)
            break
        # results[name] =

